In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras.layers import LSTM

In [3]:
d1=pd.read_csv('FakeNewsNet.csv')

In [ ]:
d1.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [44]:
d1['title'][2]

"Sophia Bush Sends Sweet Birthday Message to 'One Tree Hill' Co-Star Hilarie Burton: 'Breyton 4eva'"

In [4]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [37]:
tokenizer = Tokenizer(num_words=1000, split=' ')
tokenizer.fit_on_texts(d1['title'].values)
X = tokenizer.texts_to_sequences(d1['title'].values)
X = pad_sequences(X)

In [38]:
X

array([[  0,   0,   0, ...,   4, 231,  68],
       [  0,   0,   0, ..., 116, 152, 208],
       [  0,   0,   0, ...,   3, 319,  34],
       ...,
       [  0,   0,   0, ..., 167,   5,   1],
       [  0,   0,   0, ..., 429, 278, 130],
       [  0,   0,   0, ..., 426, 115,  33]], dtype=int32)

In [17]:
X.shape

(23196, 29)

In [18]:
X.shape[1]

29

In [40]:
model = Sequential()
model.add(Embedding(1000, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 29, 120)           120000    
                                                                 
 spatial_dropout1d_5 (Spati  (None, 29, 120)           0         
 alDropout1D)                                                    
                                                                 
 lstm_5 (LSTM)               (None, 100)               88400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 208501 (814.46 KB)
Trainable params: 208501 (814.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [41]:
y = d1['real']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [43]:
batch_size=20
model.fit(X_train, y_train, epochs = 2, verbose = 'auto')

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


580/580 [==============================] - 51s 76ms/step - loss: 0.0000e+00 - accuracy: 0.7508
Epoch 2/2
580/580 [==============================] - 47s 80ms/step - loss: 0.0000e+00 - accuracy: 0.7508


In [49]:
text =["'Sophia Bush Sends Sweet Birthday Message to 'One Tree Hill' Co-Star Hilarie Burton: 'Breyton 4eva'"]


In [65]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)  # texts is a list of input texts

# Convert the input text to sequences
sequences = tokenizer.texts_to_sequences(text)

# Pad the sequences to a fixed length (if needed)
max_sequence_length = 100  # You can adjust this to your desired sequence length
padded_sequences = pad_sequences(sequences, maxlen=X.shape[1], padding='post', truncating='post')

# Make predictions using the model
pred1 = model.predict(padded_sequences)
output_list = pred1.tolist()
print(output_list[0][0])

1/1 [==============================] - 0s 24ms/step
1.0
